In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 读取数据集
df = pd.read_csv('./updated_gas_oil_tasks .csv')  # 你的数据集路径

df = df.loc[df['general_name'].notna()]  # 删除缺少 'generalized_work_class' 的行
df = df.loc[df['task_name'].notna()]  # 删除缺少 '


In [2]:
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# 检查拆分后的数据
print(f"训练集大小: {train_data.shape[0]}")
print(f"测试集大小: {test_data.shape[0]}")

# 保存拆分后的数据集
train_data.to_csv('train_data.csv', index=False)
test_data.to_csv('test_data.csv', index=False)


训练集大小: 994
测试集大小: 249


In [3]:
train_data = pd.read_csv('train_data.csv')  # 训练数据集
test_data = pd.read_csv('test_data.csv')  # 测试数


In [4]:
from spellchecker import SpellChecker

russian = SpellChecker(language='ru', distance=1)

def correct_the_spelling(s):
    '''修正字符串中的拼写错误'''
    new_string = ''
    for i in s:
        if i not in ['свай', 'балок', 'лестниц', 'т1', 'газа', 'бурение', 'пнр', 'мк']:
            i_corr = russian.correction(i)
            if i_corr is not None:
                new_string = new_string + " " + i_corr
            else:
                new_string = new_string + " " + i
        else:
            new_string = new_string + " " + i
    return new_string

# 拼写校正
train_data['work_name_corr'] = train_data['task_name'].str.split()
train_data['work_name_corr'] = train_data['work_name_corr'].apply(correct_the_spelling)

test_data['work_name_corr'] = test_data['task_name'].str.split()
test_data['work_name_corr'] = test_data['work_name_corr'].apply(correct_the_spelling)

# 保存拼写修正后的数据
train_data.to_csv('train_data_spell_corr.csv', index=False)
test_data.to_csv('test_data_spell_corr.csv', index=False)

# 重新读取修正后的数据
train_data = pd.read_csv('train_data_spell_corr.csv')
test_data = pd.read_csv('test_data_spell_corr.csv')


In [5]:
test_data


,task_name,measurement,processed_task_name,cluster,general_name,work_name_corr
0,Монтаж защитного заземления зданий и сооружени...,м,монтаж защитного заземления зданий сооружений ...,129,заземления защитного зданий сооружений заземли...,Монтаж защитного заземления здание и сооружен...
1,Демонтаж фундаментных стаканов под колонны и о...,шт,демонтаж фундаментных стаканов колонны опоры,237,стаканов колонны опоры фундаментных демонтаж,Демонтаж фундаментных стаканов под колонна и ...
2,Осушка внутренней полости технологических труб...,км,осушка внутренней полости технологических труб...,7,осушка внутренней полости технологических труб...,Осушка внутренний полости технологических тру...
3,Предварительное пневматическое испытание трубо...,км,предварительное пневматическое испытание трубо...,4,предварительное пневматическое испытание трубо...,Предварительное пневматическое испытание труб...
4,Прокладка самонесущего кабеля ВОЛС по железобе...,м,прокладка самонесущего кабеля волс железобетон...,200,волс кабеля прокладка стальным самонесущего,Прокладка самонесущего кабель волк по железоб...
...,...,...,...,...,...,...
244,Прокладка кабеля слаботочной сети по конструкц...,м,прокладка кабеля слаботочной сети конструкциям...,56,коробах конструкциям блоках трубах сети,Прокладка кабель слаботочной сети по конструк...
245,Демонтаж ламината (полы),м2,демонтаж ламината полы,227,полы демонтаж керамогранита ламината линолеума,Демонтаж ламината (полы)
246,Промывка трубопроводов ИС без дезинфекции,м3,промывка трубопроводов ис дезинфекции,189,промывка ис дезинфекцией дезинфекции трубопров...,Промывка трубопроводов и без дезинфекции
247,Контроль качества изоляции трубопроводов метод...,км,контроль качества изоляции трубопроводов метод...,20,поляризации катодной изоляции качества методом,Контроль качества изоляция трубопроводов мето...


In [14]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# 假设 train_data 和 test_data 已经加载好
# train_data 包含 ['work_name_corr', 'measurement', 'general_name', 'standard_name']
# test_data 包含 ['task_name', 'measurement']

# 1. 基于匹配规则构建 `standard_name` 列
# 创建 global_work_class_dict（第一轮匹配），这是我们之前提到的匹配字典
global_work_class_dict = {}
for index, row in train_data.iterrows():
    key = (row['work_name_corr'], row['measurement'])
    global_work_class_dict[key] = (row['general_name'], row['standard_name'])

# 使用规则填充 `standard_name` 列，如果有匹配则填充，否则使用 `general_name`
def map_standard_name(row):
    key = (row['task_name'], row['measurement'])
    if key in global_work_class_dict:
        _, standard_name = global_work_class_dict[key]  # 提取标准名称
        return standard_name
    else:
        # 如果没有匹配，则使用 general_name
        return row['task_name']  # 默认值可以是任务名称

# 对 test_data 使用规则填充 `standard_name` 列
test_data['standard_name'] = test_data.apply(map_standard_name, axis=1)

# 2. 数据预处理：为模型准备特征和标签
# 使用 LabelEncoder 对 categorical 特征进行编码
le_task_name = LabelEncoder()
le_measurement = LabelEncoder()
le_general_name = LabelEncoder()
le_standard_name = LabelEncoder()

# 编码 train_data
train_data['work_name_corr_enc'] = le_task_name.fit_transform(train_data['work_name_corr'])
train_data['measurement_enc'] = le_measurement.fit_transform(train_data['measurement'])
train_data['general_name_enc'] = le_general_name.fit_transform(train_data['general_name'])
train_data['standard_name_enc'] = le_standard_name.fit_transform(train_data['standard_name'])

# 对 test_data 进行编码（仅对输入特征进行编码）
test_data['work_name_corr_enc'] = le_task_name.transform(test_data['task_name'])
test_data['measurement_enc'] = le_measurement.transform(test_data['measurement'])

# 3. 训练模型：预测 general_name 和 standard_name
X_first_match = train_data[['work_name_corr_enc', 'measurement_enc']]
y_first_match = train_data[['general_name_enc', 'standard_name_enc']]

# 拆分训练集和验证集（第一次匹配）
X_train, X_val, y_train, y_val = train_test_split(X_first_match, y_first_match, test_size=0.2, random_state=42)

# 使用随机森林训练模型
rf_general_name = RandomForestClassifier(n_estimators=100, random_state=42)
rf_standard_name = RandomForestClassifier(n_estimators=100, random_state=42)

# 训练模型
rf_general_name.fit(X_train, y_train['general_name_enc'])
rf_standard_name.fit(X_train, y_train['standard_name_enc'])

# 验证模型的准确率
general_name_pred = rf_general_name.predict(X_val)
standard_name_pred = rf_standard_name.predict(X_val)

print("First match accuracy (general_name):", accuracy_score(y_val['general_name_enc'], general_name_pred))
print("First match accuracy (standard_name):", accuracy_score(y_val['standard_name_enc'], standard_name_pred))

# 4. 在 test_data 上进行第一次预测
X_test = test_data[['work_name_corr_enc', 'measurement_enc']]
test_data['predicted_general_name_enc'] = rf_general_name.predict(X_test)
test_data['predicted_standard_name_enc'] = rf_standard_name.predict(X_test)

# 5. 解码预测值
test_data['predicted_general_name'] = le_general_name.inverse_transform(test_data['predicted_general_name_enc'])
test_data['predicted_standard_name'] = le_standard_name.inverse_transform(test_data['predicted_standard_name_enc'])

# 6. 计算匹配精度
# 计算第一次匹配的准确率
print("Accuracy for first match (general_name vs. predicted_general_name):", accuracy_score(test_data['general_name'], test_data['predicted_general_name']))
print("Accuracy for first match (standard_name vs. predicted_standard_name):", accuracy_score(test_data['standard_name'], test_data['predicted_standard_name']))

# 7. 保存结果
test_data.to_csv('mapped_test_data_with_model.csv', index=False)


KeyError: 'standard_name'

: 